In [ ]:
 coding: utf-8 -*-
"""
Created on Wed May  3 17:00:58 2023

@author: Breno Toledo

link do chatbot
http://t.me/BrenoToledo_testes_bot

"""

! pip install pyTelegramBotAPI

import os
import json
import telebot
from telebot import types
from datetime import datetime

BOT_TOKEN="6133902274:AAFWoP-P0IB80mjZ0JbgE87pq77exJdpT-I"

bot = telebot.TeleBot(BOT_TOKEN)

NAME_DB_OCORRENCIAS = "ocorrencias"
NAME_DB_TECNICOS = "tecnicos"
# ------------------------------------------------------------------------MODEL
def save_data(data,name_database):
    
    # Define o File
    file = f"{name_database}.json"
    
    # Carregar dados antigos do arquivo, se houver
    if os.path.exists(file):
        with open(file, 'r') as f:
            old_data = json.load(f)
    else:
        old_data = []
    
    # Adicionar novos dados à lista
    old_data.append(data)
    
    
    
    # Salvar a lista atualizada no arquivo
    with open(file, 'w') as f:
        json.dump(old_data, f)

def get_today_data(name_database):
    
    # Define o File
    file = f"{name_database}.json"
    
    # Carregar dados antigos do arquivo, se houver
    if os.path.exists(file):
        with open(file, 'r') as f:
            all_data = json.load(f)
    else:
        all_data = []
    
    # Filtrar os dados para obter apenas as ocorrências de hoje
    today = datetime.now().strftime("%Y-%m-%d")
    today_data = [data for data in all_data if data.get('date') == today]
    
    return today_data

def get_local_emoji(local):
    if local == "\udc3e SCHAAF":
        return "🌽"
    elif local == "\udc2e TODDY":
        return "🐮"

def get_status_emoji(status):
    if status == "\ud83d\udfe2 RESOLVIDO":
        return "🟢"
    elif status == "\ud83d\udff4️ PARADO":
        return "🔴"
    elif status == "\ud83d\udfe1 PRÓXIMO TURNO":
        return "🟡"
    elif status == "\ud83d\udfe0 ENGEHARIA":
        return "🟠"

def get_location(message, data):

    # Adicionar a resposta do usuário aos dados existentes
    data['ocorrencia'] = message.text.lower()

    # Perguntar sobre o local do problema
    markup = types.ReplyKeyboardMarkup(row_width=1, one_time_keyboard=True)
    itembtn1 = types.KeyboardButton('🌽 SCHAAF')
    itembtn2 = types.KeyboardButton('🐮 TODDY')
    markup.add(itembtn1, itembtn2)
    bot.send_message(message.chat.id, "Qual é o local do problema?", reply_markup=markup)
    bot.register_next_step_handler(message, process_location, data)

    #print(f"get_location :{data}")
    
def process_location(message, data):
    # Adicionar a resposta do usuário aos dados existentes
    data['location'] = message.text.lower()
    
    # Perguntar se houve um downtime
    markup = types.ReplyKeyboardMarkup(row_width=2, one_time_keyboard=True)
    itembtn1 = types.KeyboardButton('Sim')
    itembtn2 = types.KeyboardButton('Não')
    markup.add(itembtn1, itembtn2)
    bot.send_message(message.chat.id, "Houve um downtime?", reply_markup=markup)
    bot.register_next_step_handler(message, process_downtime, data)

    #print(f"process_location :{data}")

def process_downtime(message, data):
    # Adicionar a resposta do usuário aos dados existentes
    data['downtime'] = message.text.lower() == 'sim'
    
    if data['downtime']:
        # Perguntar quanto tempo foi o downtime
        bot.send_message(message.chat.id, "Quanto tempo foi o downtime (em minutos)?")
        bot.register_next_step_handler(message, process_downtime_duration, data)
    else:
        # Perguntar sobre a existência de uma ordem de serviço
        markup = types.ReplyKeyboardMarkup(row_width=2, one_time_keyboard=True)
        itembtn1 = types.KeyboardButton('Sim')
        itembtn2 = types.KeyboardButton('Não')
        markup.add(itembtn1, itembtn2)
        bot.send_message(message.chat.id, "Existe uma ordem de serviço?", reply_markup=markup)
        bot.register_next_step_handler(message, process_order, data)

    #print(f"process_downtime :{data}")

def process_downtime_duration(message, data):
    # Adicionar a resposta do usuário aos dados existentes
    data['downtime_duration'] = message.text
    
    # Perguntar sobre a existência de uma ordem de serviço
    markup = types.ReplyKeyboardMarkup(row_width=2, one_time_keyboard=True)
    itembtn1 = types.KeyboardButton('Sim')
    itembtn2 = types.KeyboardButton('Não')
    markup.add(itembtn1, itembtn2)
    bot.send_message(message.chat.id, "Existe uma ordem de serviço?", reply_markup=markup)
    bot.register_next_step_handler(message, process_order, data)

    #print(f"process_downtime_duration :{data}")

def process_order(message, data):
    # Adicionar a resposta do usuário aos dados existentes
    data['order'] = message.text.lower() == 'sim'
    
    if data['order']:
        # Perguntar qual é o número da ordem de serviço
        bot.send_message(message.chat.id, "Qual é o número da ordem de serviço?")
        bot.register_next_step_handler(message, process_order_number, data)
    else:
        # Informar que a ordem de serviço não será utilizada
        
        # Perguntar se o problema foi resolvido ou será passado para o próximo turno
        markup = types.ReplyKeyboardMarkup(row_width=1, one_time_keyboard=True)
        itembtn1 = types.KeyboardButton('🟢 RESOLVIDO')
        itembtn2 = types.KeyboardButton('🔴 PARADO')
        itembtn3 = types.KeyboardButton('🟡 PRÓXIMO TURNO')
        itembtn4 = types.KeyboardButton('🟠 ENGENHARIA')
        itembtn5 = types.KeyboardButton('⭐ MELHORIA')
        markup.add(itembtn1, itembtn2, itembtn3, itembtn4, itembtn5)
        bot.send_message(message.chat.id, "Qual o status do problema?", reply_markup=markup)
        bot.register_next_step_handler(message, process_problem, data)


    print(f"process_order :{data}")

def process_order_number(message, data):

    # Adicionar a resposta do usuário aos dados existentes
    data['ordem_numero'] = message.text 

    # Perguntar se o problema foi resolvido ou será passado para o próximo turno
    markup = types.ReplyKeyboardMarkup(row_width=1, one_time_keyboard=True)
    itembtn1 = types.KeyboardButton('🟢 RESOLVIDO')
    itembtn2 = types.KeyboardButton('🔴 PARADO')
    itembtn3 = types.KeyboardButton('🟡 PRÓXIMO TURNO')
    itembtn4 = types.KeyboardButton('🟠 ENGENHARIA')
    itembtn5 = types.KeyboardButton('⭐ MELHORIA')
    markup.add(itembtn1, itembtn2, itembtn3, itembtn4, itembtn5)
    bot.send_message(message.chat.id, "Qual o status do problema?", reply_markup=markup)
    bot.register_next_step_handler(message, process_problem, data)


def process_problem(message, data):
    # Salvar a resposta do usuário em um arquivo JSON

    data['status'] = message.text

    #print(f"process_status :{data}")

    save_data(data, NAME_DB_OCORRENCIAS)
    
    bot.reply_to(message, "Obrigado, o problema foi registrado!")

@bot.message_handler(commands=['start', 'oi'])
def send_welcome(message):
    markup = types.ReplyKeyboardMarkup(row_width=2, one_time_keyboard=True)
    itembtn1 = types.KeyboardButton('Inserir ocorrência')
    itembtn2 = types.KeyboardButton('Exportar Relatório')
    markup.add(itembtn1, itembtn2)
    bot.reply_to(message, "Como posso te ajudar ?", reply_markup=markup)

@bot.message_handler(func=lambda msg: True)
def echo_all(message):


    if message.text == "Inserir ocorrência":
        # Criar data
        data = {
        'user_id': message.chat.id,
        'date': datetime.now().strftime("%Y-%m-%d")
        }
        # Perguntar qual foi o problema
        bot.send_message(message.chat.id, "Qual o problema ocorrido?")
        bot.register_next_step_handler(message, get_location, data)
    elif message.text == "Exportar Relatório":
        # Obter as ocorrências de hoje
        today_data = get_today_data(NAME_DB_OCORRENCIAS)

        if today_data:
            # Criar uma lista formatada de ocorrências
            message_text = f'📅 *DATA:* \n```{datetime.now().strftime("%d/%m/%Y")}```\n\n🐗 *EQUIPE: 1º TURNO* \n\nBreno\nDanilo\nAnderson\nTomas\n\n*👉LEGENDA:*\n\n🚩 DOWNTIME\n🟢 RESOLVIDO\n🔴 PARADO\n🟡 PRÓXIMO TURNO\n🟠 ENGENHARIA\n⭐ MELHORIA\n\n'
            ocorrencias_por_local = {}
            for data in today_data:
                if data["location"] not in ocorrencias_por_local:
                    ocorrencias_por_local[data["location"]] = []
                ocorrencias_por_local[data["location"]].append(data)
            for local, ocorrencias in ocorrencias_por_local.items():
                message_text += f'*{local.upper()}*\n\n'
                for ocorrencia in ocorrencias:
                    message_text += f'\n\n{ocorrencia["status"][0]} {ocorrencia["ocorrencia"]}\n\n'
                    if ocorrencia["downtime"] == True :
                        message_text += f'🚩 {ocorrencia["downtime_duration"]} Min'
                        
                    if ocorrencia["order"] == True :
                        message_text += f' - Ordem: {ocorrencia["ordem_numero"]}'
                message_text += '\n\n'
            bot.reply_to(message, message_text)
        else:
            message_text = "Não houve ocorrências registradas hoje."
            bot.reply_to(message, message_text)
    else:
        bot.reply_to(message, message.text)


bot.infinity_polling()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.4/230.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.11.0-py3-none-any.whl size=212399 sha256=c716305068258cc38ef48375bd45ea7c024c87ccfde9abc807a2eaf6b488dbf8
  Stored in directory: /root/.cache/pip/wheels/ee/54/14/73e9f99d3a4c02df8d106566f93bd29d41ab7f3915cc13a23d
Successfully built pyTelegramBotAPI


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



```
! pip install pyTelegramBotAPI

import os
import json
import telebot
from telebot import types
from datetime import datetime

bot = telebot.TeleBot(BOT_TOKEN)

def save_data(data):
    # Carregar dados antigos do arquivo, se houver
    if os.path.exists('data.json'):
        with open('data.json', 'r') as f:
            old_data = json.load(f)
    else:
        old_data = []
    
    # Adicionar novos dados à lista
    old_data.append(data)
    
    # Salvar a lista atualizada no arquivo
    with open('data.json', 'w') as f:
        json.dump(old_data, f)

def get_today_data():
    # Carregar dados antigos do arquivo, se houver
    if os.path.exists('data.json'):
        with open('data.json', 'r') as f:
            all_data = json.load(f)
    else:
        all_data = []
    
    # Filtrar os dados para obter apenas as ocorrências de hoje
    today = datetime.now().strftime("%Y-%m-%d")
    today_data = [data for data in all_data if data.get('date') == today]
    
    return today_data

@bot.message_handler(commands=['start', 'hello'])
def send_welcome(message):
    markup = types.ReplyKeyboardMarkup(row_width=1, one_time_keyboard=True)
    itembtn1 = types.KeyboardButton('Inserir ocorrência')
    itembtn2 = types.KeyboardButton('Exportar Relatório')
    markup.add(itembtn1, itembtn2)
    bot.reply_to(message, "Howdy, how are you doing?", reply_markup=markup)

@bot.message_handler(func=lambda msg: True)
def echo_all(message):
    if message.text == "Inserir ocorrência":
        # Perguntar qual foi o problema
        bot.send_message(message.chat.id, "Qual o problema ocorrido?")
        bot.register_next_step_handler(message, process_problem)
    elif message.text == "Exportar Relatório":
        # Obter as ocorrências de hoje
        today_data = get_today_data()
        
        if today_data:
            # Criar uma lista formatada de ocorrências
            message_text = "Ocorrências de hoje:\n"
            for data in today_data:
                message_text += f"\nUsuário: {data['user_id']}\nProblema: {data['problem']}\nLocal: {data.get('location', '')}\nDowntime: {data.get('downtime', '')}\nOrdem de Serviço: {data.get('os_number', '')}\nResolvido: {data.get('resolved', '')}\n"
        else:
            message_text = "Não houve ocorrências registradas hoje."
        
        # Enviar a lista de ocorrências
        bot.reply_to(message, message_text)
    else:
        bot.reply_to(message, message.text)

def process_problem(message):
    # Salvar a resposta do usuário em um arquivo JSON
    data = {
        'user_id': message.chat.id,
        'problem': message.text,
        'date': datetime.now().strftime("%Y-%m-%d")
    }
    
    # Perguntar qual é o local
   

```



BOT_TOKEN="1887737871:AAF7DKL_rj7F_o1TA2vzx6gHPXgtpgGccPo"
